## With this notebook; 

* I want to keep up my DataCamp Tracks, I created the dataset by the courses which I am interested in individually including track information and put dataset on my Google Drive acoount. For reading Google spreadsheet; after create project and Api Authentication(Google Sheet API and Google Drive API), I started to read from drive excel.

* My goal is to follow up on this file and work on the current file instead of a fixed dataset so that I can follow my progress.
* While I was developing my Data Science skills with the DataCamp course, this training time also will be a project for me :)

In [1]:
import gspread

gc=gspread.oauth()
sheet1 = gc.open("datacamp").sheet1

data = sheet1.get_all_values()
headers = data.pop(0)


In [2]:

import pandas as pd
df = pd.DataFrame(data, columns=headers)

** pre_processing data, NaN values, dtypes **

In [3]:
df.replace('', 0, inplace=True)
df['Planning Date']=pd.to_datetime(df['Planning Date'])
col_dict={'No':int, 'order':int,'Watch':int,'Learn':int,'Github':int,'Video':int,'Pdf':int}
for k, v in col_dict.items():
    df[k]=df[k].astype(v)
print(df.dtypes)

No                        int32
Track                    object
Type                     object
order                     int32
Course                   object
Planning Date    datetime64[ns]
Watch                     int32
Learn                     int32
Github                    int32
Video                     int32
Pdf                       int32
dtype: object


** define necessary methods for statisticals **

In [4]:
def remain(x):
    return x.count() - x.sum()
remain.__name__ = 'Remain'

def percent(x):
    return x.sum()*100 / x.count()
percent.__name__='Comp. %'

** some courses take place in multiple tracks. Once you complete one, others must be update as completed  as well. **

In [5]:
from collections import Counter
course_count= Counter(df.Course)
for course in course_count.most_common():
    print(course)


('Python Data Science Toolbox (Part 1)', 5)
('Python Data Science Toolbox (Part 2)', 5)
('Intermediate Python', 5)
('Data Manipulation with pandas', 4)
('Writing Efficient Python Code', 4)
('Writing Functions in Python', 4)
('Introduction to Data Visualization with Seaborn', 4)
('Introduction to Python', 4)
('Supervised Learning with scikit-learn', 4)
('Unsupervised Learning in Python', 4)
('Machine Learning with Tree-Based Models in Python', 4)
('Working with Dates and Times in Python', 3)
('Introduction to Shell', 3)
('Unit Testing for Data Science in Python', 3)
('Object-Oriented Programming in Python', 3)
('Joining Data with pandas', 3)
('Introduction to Data Visualization with Matplotlib', 3)
('Introduction to Importing Data in Python', 3)
('Intermediate Importing Data in Python', 3)
('Cleaning Data in Python', 3)
('Analyzing Police Activity with pandas', 3)
('Introduction to Relational Databases in SQL', 3)
('Statistical Thinking in Python (Part 1)', 3)
('Cluster Analysis in Pyth

In [6]:
update_columns={'Watch':'max', 'Learn':'max', 'Github':'max', 'Video':'max','Pdf':'max','Planning Date':'min'}
max_=df.groupby('Course').agg(update_columns)
for key, row_ in zip(max_.index, max_.values):
    df.loc[df.Course==key, update_columns.keys()]=row_

** summary table for completence of the course, skill tracks **

In [7]:
statistics={'Learn':['sum','count', remain, percent], 
                         'Github':'sum', 
                         'Video':'sum',
                         'Pdf':'sum'}
df.sort_values(['Type','No','Track'],ascending=False).groupby(['Type','No','Track']).agg(statistics)

Learn               \
                                                   sum count Remain   
Type   No Track                                                       
Career 1  Python Programmer                          2    16     14   
       2  Data Analyst with Python                   1    16     15   
       3  Data Scientist with Python                 2    23     21   
       4  Data Science for Everyone                  2    26     24   
       5  Machine Learning for Everyone              1    29     28   
       6  Machine Learning Scientist with Python     0    23     23   
       7  Data Engineer with Python                  0    25     25   
Course 18 Natural Language Processing in Python      0     2      2   
       19 Marketing Analytics with Python            0     2      2   
       20 Big Data with PySpark                      0     1      1   
          Finance Fundamentals in Python             0     1      1   
          Statistics Fundamentals with Python        0     1      1   
          Tek                                        0    11     11   
Skill  1  Python Toolbox                             0     5      5   
       2  Python Programming                         0     6      6   
       3  Data Visualization with Python             0     5      5   
       4  Python Fundamentals                        1     4      3   
       5  Deep Learning for NLP in Python            0     3      3   
       6  SQL for Business Analysts                  0     5      5   
       7  Applied Finance in Python                  0     4      4   

                                                            Github Video Pdf  
                                                    Comp. %    sum   sum sum  
Type   No Track                                                               
Career 1  Python Programmer                       12.500000      4     0   0  
       2  Data Analyst with Python                 6.250000      1     0   0  
       3  Data Scientist with Python               8.695652      4     0   0  
       4  Data Science for Everyone                7.692308      2     0   0  
       5  Machine Learning for Everyone            3.448276      1     0   0  
       6  Machine Learning Scientist with Python   0.000000      0     0   0  
       7  Data Engineer with Python                0.000000      1     0   0  
Course 18 Natural Language Processing in Python    0.000000      0     0   0  
       19 Marketing Analytics with Python          0.000000      0     0   0  
       20 Big Data with PySpark                    0.000000      0     0   0  
          Finance Fundamentals in Python           0.000000      0     0   0  
          Statistics Fundamentals with Python      0.000000      0     0   0  
          Tek                                      0.000000      0     0   0  
Skill  1  Python Toolbox                           0.000000      1     0   0  
       2  Python Programming                       0.000000      1     0   0  
       3  Data Visualization with Python           0.000000      0     0   0  
       4  Python Fundamentals                     25.000000      1     0   0  
       5  Deep Learning for NLP in Python          0.000000      0     0   0  
       6  SQL for Business Analysts                0.000000      0     0   0  
       7  Applied Finance in Python                0.000000      0     0   0

** This is dictionary of career tracks including courses with order number **

In [8]:
career_tracks= list(map( lambda Track :{Track: dict(zip(df[df['Track']==Track].order, df[df['Track']==Track]['Course']))}, df[df['Type']=='Career'].Track))

In [9]:
for career in career_tracks[:5]:
    print(career)

{'Python Programmer': {1: 'Introduction to Data Science in Python', 2: 'Data Types for Data Science in Python', 3: 'Data Manipulation with pandas', 4: 'Python Data Science Toolbox (Part 1)', 5: 'Python Data Science Toolbox (Part 2)', 6: 'Writing Efficient Python Code', 7: 'Working with Dates and Times in Python', 8: 'Regular Expressions in Python', 9: 'Web Scraping in Python<', 10: 'Writing Functions in Python', 11: 'Introduction to Shell', 12: 'Conda Essentials', 13: 'Parallel Programming with Dask in Python', 14: 'Software Engineering for Data Scientists in Python', 15: 'Unit Testing for Data Science in Python', 16: 'Object-Oriented Programming in Python'}}
{'Python Programmer': {1: 'Introduction to Data Science in Python', 2: 'Data Types for Data Science in Python', 3: 'Data Manipulation with pandas', 4: 'Python Data Science Toolbox (Part 1)', 5: 'Python Data Science Toolbox (Part 2)', 6: 'Writing Efficient Python Code', 7: 'Working with Dates and Times in Python', 8: 'Regular Expre

In [10]:

from collections import Counter
learn_count= Counter(df.Learn)
for course in learn_count.most_common(5):
    print('.',course,'.')

. (0, 199) .
. (1, 9) .
